<a href="https://colab.research.google.com/github/raphaelp-silva/deep_learning_com_pytorch_e_python/blob/main/Projeto_7_Classifica%C3%A7%C3%A3o_multiclasse_iris_com_valida%C3%A7%C3%A3o_cruzada_com_dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 7: Classificação multiclasse iris com validação cruzada

## Etapa 1: Importação das bibliotecas

In [1]:
!pip install skorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 2.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
from sklearn.model_selection import cross_val_score
torch.__version__

'2.5.1+cu124'

## Etapa 2: Base de dados

In [3]:
np.random.seed(123)
torch.manual_seed(123)

In [5]:
base = pd.read_csv('iris.csv')
previsores = base.iloc[:, 0:4].values
classe = base.iloc[:, 4].values

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
classe = encoder.fit_transform(classe)

In [7]:
np.unique(classe)

array([0, 1, 2])

In [8]:
previsores = previsores.astype('float32')
classe = classe.astype('int64')

In [9]:
type(previsores)

numpy.ndarray

## Etapa 3: Construção do modelo

In [10]:
import torch
import torch.nn as nn

class ClassificadorTorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(4, 16)
        self.activation0 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.dense1 = nn.Linear(16, 16)
        self.activation1 = nn.ReLU()
        self.dense2 = nn.Linear(16, 3)

        self.output = nn.Softmax(dim=1) # usando softmax para classificação multiclasse

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation0(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation1(X)
        X = self.dense2(X)
        X = self.output(X)  # utilizando softmax para classificação multiclasse
        return X


In [12]:
classificador_sklearn = NeuralNetClassifier(module = ClassificadorTorch,
                                            criterion = torch.nn.CrossEntropyLoss,
                                            optimizer = torch.optim.Adam,
                                            max_epochs = 1000,
                                            batch_size = 10,
                                            train_split = False)

## Etapa 4: Validação cruzada

In [13]:
resultados = cross_val_score(classificador_sklearn, previsores, classe, cv = 5,
                             scoring = 'accuracy')

A saída de streaming foi truncada nas últimas 5000 linhas.
      9        0.8927  0.0196
     10        0.8828  0.0202
     11        0.8574  0.0197
     12        0.8391  0.0202
     13        0.8658  0.0205
     14        0.8496  0.0214
     15        0.8538  0.0193
     16        0.8418  0.0194
     17        0.8208  0.0197
     18        0.8481  0.0200
     19        0.8036  0.0207
     20        0.7964  0.0198
     21        0.7619  0.0197
     22        0.8519  0.0198
     23        0.8098  0.0193
     24        0.7023  0.0226
     25        0.8701  0.0250
     26        0.8216  0.0220
     27        0.7204  0.0197
     28        0.7442  0.0194
     29        0.7096  0.0189
     30        0.7016  0.0215
     31        0.6942  0.0204
     32        0.7271  0.0209
     33        0.6562  0.0197
     34        0.7512  0.0215
     35        0.7258  0.0193
     36        0.7355  0.0179
     37        0.6952  0.0218
     38        0.6775  0.0178
     39        0.6998  0.0193
     40    

In [14]:
media = resultados.mean()
desvio = resultados.std()

In [15]:
media, desvio

(0.9666666666666666, 0.04216370213557838)

In [17]:
resultados

array([1.        , 1.        , 0.9       , 0.93333333, 1.        ])